In [4]:
import _pickle as cpickle
import xarray as xr
import pandas as pd
import numpy as np

In [51]:
import itertools

In [2]:
import sys
sys.path.append('../scripts/indices')

In [3]:
sys.path.append("/home/abhi/Documents/mygit/postBC_diagnostic/src/scripts")
from recipes import *

In [5]:
import prInd

In [87]:
import tasInd

In [6]:
outdir = '/home/abhi/Documents/mygit/NEX-Analysis/pickles/Amravati/Indices'

In [8]:
scenIndex = pd.Index(['historical', 'rcp45', 'rcp85'], name='scen')

### Load dsets

In [40]:
dsets = {}

In [41]:
fut_years = range(2006, 2090 + 1)

In [43]:
for scen in scenIndex:
    with open(f'../../pickles/Amravati/Amravati_ALL21_{scen}_ds.pkl', 'rb') as f:
        if scen in ['rcp45', 'rcp85']:
            dsets[scen] = selyear(cpickle.load(f), fut_years)
        else:
            dsets[scen] = cpickle.load(f)
            

In [44]:
def extract_var(scen, variable, ens_mean=True):
    ds = dsets[scen]
    var = ds.sel(variable=variable)['value']
    
    if variable == 'pr':
        var = var*86400
        units = 'mm/day'
    elif variable in ['tasmax', 'tasmin']:
        var = var - 273.15
        units = 'degC'
        
        
    if ens_mean:
        var = var.mean(dim='model')
    
    var.attrs['units'] = units

        
    return(var)

### make_metric_ds

In [45]:
thresh_vals, ndays = [1], [3, 5, 7, 10]
pctls = [75, 90, 95, 99]

In [46]:
base_years = range(1976, 2005 + 1)

In [47]:
def make_metric_dict_yearmean(metric_dict):
    for k, v in metric_dict.items():
        try:
            metric_dict[k] = yearmean(v)
        except KeyError:
            pass
    
    return(metric_dict)

In [108]:
def change_metric_dict_variable_coord_temperature(metric_dict):
    for k, v in metric_dict.items():
        try:
            metric_dict[k] = v.assign_coords(variable='temperature')
        except KeyError:
            pass
    
    return(metric_dict)

In [114]:
def make_metric_ds(scen, variable):
    metric_dict = {}
    
    if variable == 'precipitation':
        thresh_vals, ndays = [1], [3, 5, 7, 10]
        pctls = [75, 90, 95, 99]
        
        var = extract_var(scen, 'pr')

        for thresh_val in thresh_vals:
            thresh = f'{thresh_val} mm/day'
            metric_dict[f'max_cdd_{thresh_val}'] = prInd.max_consec_days(var, 'cdd', thresh)
            metric_dict[f'max_cwd_{thresh_val}'] = prInd.max_consec_days(var, 'cwd', thresh)
            metric_dict[f'intensity_above_{thresh_val}mm'] = prInd.intensity_above_thresh(var, thresh)
            metric_dict[f'days_above_{thresh_val}mm'] = prInd.days_above_thresh(var, thresh)
            
        for thresh_val, nday in itertools.product(thresh_vals, ndays):
            thresh = f'{thresh_val} mm/day'
            metric_dict[f'num_cdd_{thresh_val}_{nday}'] = prInd.num_consec_days(var, 'cdd', thresh, nday)
            metric_dict[f'num_cwd_{thresh_val}_{nday}'] = prInd.num_consec_days(var, 'cwd', thresh, nday)
            
        for nday in [1] + ndays:
            metric_dict[f'max_{nday}_day_pr_amt'] = prInd.max_n_day_pr_amt(var, nday)
        
        if scen == 'historical':
            var_fut = var_base = var
        else:
            var_fut = var
            var_base = extract_var('historical', 'pr')
            
        for pctl in pctls:
            metric_dict[f'intensity_above_{pctl}p'] = prInd.intensity_above_pctl(var_fut, 
                                                                                 selyear(var_base, base_years), 
                                                                                 pctl)
            metric_dict[f'days_above_{pctl}p'] = prInd.days_above_pctl(var_fut, 
                                                                       selyear(var_base, base_years), 
                                                                       pctl)
            
    elif variable == 'temperature':
        thresh_vals, ndays = [35, 40, 45], [3, 5, 7, 10]
        pctls = [75, 90, 95, 99]
        
        var = extract_var(scen, 'tasmax')

        for thresh_val in thresh_vals:
            thresh = f'{thresh_val} degC'
            metric_dict[f'max_chd_{thresh_val}'] = tasInd.max_consec_days(var, 'chd', thresh)
            metric_dict[f'intensity_above_{thresh_val}C'] = tasInd.intensity_above_thresh(var, thresh)
            metric_dict[f'days_above_{thresh_val}C'] = tasInd.days_above_thresh(var, thresh)
            
        
        for thresh_val, nday in itertools.product(thresh_vals, ndays):
            thresh = f'{thresh_val} degC'
            metric_dict[f'num_chd_{thresh_val}_{nday}'] = tasInd.num_consec_days(var, 'chd', thresh, nday)
            
            
        for nday in [1] + ndays:
            metric_dict[f'max_{nday}_day_tas_mean'] = tasInd.max_n_day_tas_mean(var, nday)
            
            
        if scen == 'historical':
            var_fut = var_base = var
        else:
            var_fut = var
            var_base = extract_var('historical', 'tasmax')
            
            
        for pctl in pctls:
            metric_dict[f'intensity_above_{pctl}p'] = tasInd.intensity_above_pctl(var_fut, 
                                                                                 selyear(var_base, base_years), 
                                                                                 pctl)
            metric_dict[f'days_above_{pctl}p'] = tasInd.days_above_pctl(var_fut, 
                                                                       selyear(var_base, base_years), 
                                                                       pctl)
            
        
        tasmax = extract_var(scen, 'tasmax')
        tasmin = extract_var(scen, 'tasmin')
        
        metric_dict['daily_temperature_range'] = tasInd.daily_temperature_range(tasmax, tasmin)
        metric_dict['daily_temperature_range_variability'] = tasInd.daily_temperature_range_variability(tasmax, tasmin)
        
        
    metricIndex = pd.Index(metric_dict.keys(), name='metric')
    metric_dict = make_metric_dict_yearmean(metric_dict)
    metric_dict = change_metric_dict_variable_coord_temperature(metric_dict)
    metric_ds = xr.concat(metric_dict.values(), dim=metricIndex)        
        
    return(metric_ds)
            
        
           

In [115]:
res = make_metric_ds('historical', 'temperature')

In [113]:
res

{'max_chd_35': <xarray.DataArray 'value' (seas: 5, year: 56)>
 array([[113., 109., 112., ..., 116., 109., 121.],
        [  0.,   0.,   0., ...,   0.,   2.,   0.],
        [ 91.,  88.,  86., ...,  91.,  89.,  91.],
        [ 22.,  21.,  26., ...,  25.,  20.,  30.],
        [  0.,   0.,   0., ...,   0.,   0.,   0.]])
 Coordinates:
     variable  <U11 'temperature'
   * seas      (seas) object 'Annual' 'DJF' 'MAM' 'JJAS' 'ON'
   * year      (year) int64 1950 1951 1952 1953 1954 ... 2001 2002 2003 2004 2005,
 'intensity_above_35C': <xarray.DataArray 'value' (seas: 5, year: 56)>
 array([[39.591107, 39.37247 , 39.380762, ..., 39.859529, 39.183538, 39.962283],
        [      nan,       nan,       nan, ...,       nan, 35.164604,       nan],
        [39.948085, 39.521384, 39.860628, ..., 40.276588, 40.451562, 40.231098],
        [38.520172, 38.925729, 37.941164, ..., 38.608354, 39.398401, 39.155839],
        [      nan,       nan,       nan, ...,       nan,       nan,       nan]])
 Coordinates

In [74]:
def get_tslice(year):
    if year in range(1976, 2006):
        tslice = 'baseline'
    elif year < 1976:
        tslice = 'baseline-remain'
    elif year in range(2006, 2021):
        tslice = '2006-2020'
    elif year in range(2021, 2051):
        tslice = 'near'
    elif year in range(2051, 2061):
        tslice = '2051-2061'
    elif year in range(2061, 2091):
        tslice = 'far'
    elif year > 2090:
        tslice = 'end-of-century'
        
    return(tslice)

In [116]:
metric_ds_base = make_metric_ds('historical', 'temperature')

In [117]:
metric_df_base = metric_ds_base.to_dataframe('value').reset_index()
metric_df_base['scen'] = 'historical'

In [118]:
metric_ds_fut = xr.concat([make_metric_ds(scen, 'temperature') for scen in scenIndex], dim=scenIndex)

In [119]:
metric_df_fut = metric_ds_fut.to_dataframe('value').reset_index()

In [120]:
metric_df = pd.concat([metric_df_base, metric_df_fut], axis=0)
metric_df['tslice'] = metric_df['year'].apply(get_tslice)


In [121]:
metric_df.to_csv(f'{outdir}/metric_df_temperature.csv', index=False)

In [123]:
metric_df.loc[metric_df.metric == 'max_chd_40']

,metric,scen,seas,value,variable,year,tslice
840,max_chd_40,historical,Annual,57.0,temperature,1950,baseline-remain
841,max_chd_40,historical,Annual,46.0,temperature,1951,baseline-remain
842,max_chd_40,historical,Annual,60.0,temperature,1952,baseline-remain
843,max_chd_40,historical,Annual,52.0,temperature,1953,baseline-remain
844,max_chd_40,historical,Annual,54.0,temperature,1954,baseline-remain
845,max_chd_40,historical,Annual,54.0,temperature,1955,baseline-remain
846,max_chd_40,historical,Annual,54.0,temperature,1956,baseline-remain
847,max_chd_40,historical,Annual,57.0,temperature,1957,baseline-remain
848,max_chd_40,historical,Annual,50.0,temperature,1958,baseline-remain
849,max_chd_40,historical,Annual,60.0,temperature,1959,baseline-remain


,metric,scen,seas,value,variable,year,tslice
0,max_chd_35,historical,Annual,113.000000,temperature,1950,baseline-remain
1,max_chd_35,historical,Annual,109.000000,temperature,1951,baseline-remain
2,max_chd_35,historical,Annual,112.000000,temperature,1952,baseline-remain
3,max_chd_35,historical,Annual,114.000000,temperature,1953,baseline-remain
4,max_chd_35,historical,Annual,106.000000,temperature,1954,baseline-remain
5,max_chd_35,historical,Annual,107.000000,temperature,1955,baseline-remain
6,max_chd_35,historical,Annual,110.000000,temperature,1956,baseline-remain
7,max_chd_35,historical,Annual,114.000000,temperature,1957,baseline-remain
8,max_chd_35,historical,Annual,107.000000,temperature,1958,baseline-remain
9,max_chd_35,historical,Annual,109.000000,temperature,1959,baseline-remain
